# Homework 4
## Giacomo Menegatti 2122852

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Finding the eccentricity e for the maximum angular deceleration at $f=\frac{\pi}{4}$
 
The equations used in this problem are $$r^2\dot f = h$$ $$r=\frac{p}{1+e\cos f} \implies \dot r = \frac{he}{p}\sin f$$
To find the maximum angular deceleration $-\ddot f$ I derive $\dot f$ twice (with $h$, $p$ and $e$ constants) obtaining $$\dot f = \frac{h}{r^2} = \frac{h}{p^2}(1+e\cos f)^2$$
$$ \ddot f= -2\frac{h^2e}{p^4}(1+e \cos f)^3\sin f$$
$$ \frac{d^3f}{dt^3}=-2\frac{h^2e}{p^4}(1+e\cos f)^2[-3e\sin^2 f + (1+e \cos f)\cos f]\dot f $$
Imposing $[...] = 0$, I obtain the equation $$e= \frac{\cos f}{3-4\cos^2 f}$$
Using $f=\frac{\pi}{4}$, I get $e=0.7071$

## Laplace to Heliocentric RF transformation
### Read the orbital elements from the file

In [27]:
# Read the file skipping the comments and the last column (;)
df = pd.read_csv('Elementi Orbitali (Meeus - MEE of date).m', delimiter='\s+', comment='%', header=None, usecols=[0,1,2,3])

planet_names = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']
planet = {}
for i, name in enumerate(planet_names):
  #Save the orbital elements in a dictionary. I pick blocks of 7 rows at every time because the last one is blank
  orbital_elements = {}
  orbital_elements['L'] = np.array(df.iloc[7*i])
  orbital_elements['a'] = np.array(df.iloc[7*i+1])
  orbital_elements['e'] = np.array(df.iloc[7*i+2])
  orbital_elements['i'] = np.array(df.iloc[7*i+3])
  orbital_elements['Omega'] = np.array(df.iloc[7*i+4])
  orbital_elements['pi'] = np.array(df.iloc[7*i+5])

  #Save the dictionary with the planet name
  planet[name]=orbital_elements


### Get the Julian time at a specific date
For a given date Day-Month-Year hour the Julian time is given by 
$$JD_0 = D – 32075 + 1461 × (Y + 4800 + (M – 14)/12)/4
+ 367 × (M – 2 – (M – 14)/12 × 12)/12
–3 × ((Y + 4900 + (M – 14) / 12) / 100) /4$$
$$JD = JD_0 + F – 0.5$$
where $JD_0$ is the Julian date at noon and $JD$ the date at any fraction of day $F$ from the midnight.

In [28]:
def JD(Y=2000, M=1, D=1, h=0, m=0, s=0, TZ=0):
  '''Calculate the Julian Time from the Gregorian date'''
  # In python trunc keeps only the integer part. //, round and floor do not work with negative nubers
  C = np.trunc((M-14)/12)
  JD0 = D-32075 + np.trunc(1461*(Y+4800+C)/4) + np.trunc(367*(M-2-C*12)/12) - np.trunc(3*np.trunc((Y+4900+C)/100)/4)
  # Add the hour information and the Time Zone
  JD = JD0 - 0.5 + TZ/24 + (h+m/60+s/3600)/24
  return JD


Define T, GMST and $\epsilon$

In [29]:
def T(JD):
  '''Return the time in centuries from J2000.0'''
  return (JD-2451545.0)/36525

def eps(T):
  '''Inclination of the Earth axis in RADIANS'''
  eps = 23.439291 - 0.0130042*T - 0.00059*T**2 + 0.001813*T**3
  return np.deg2rad(eps)

def GMST(Y, D, M, h=0, m=0, s=0, TZ=-1):
  '''Greenwich Mean Sidereal Time'''
  T = T(JD(Y,M,D))  #The Julian date is calculated from the midnight at UT1=0
  GMST0 = 24110.54841 + 8640184.812866*T + 0.093104*T**2 - 0.0000062*T**3 #GMST at UT1=0
  F = (h + m/60 + s/3600)/24 + TZ/24  #Time from UT1 in fraction of days
  GMST = GMST0 + F*24*3600*1.002737909350795 #GMST at the right time calculated in seconds
  return GMST*2*np.pi/(24*3600) #GMST angle in RADIANS



### Cartesian Position of a planet at a given date and time

In [30]:
def Rx(theta, v):
  '''Rotation of a vector along the x-axis'''
  return np.array(v[0], v[1]*np.cos(theta)+v[2]*np.sin(theta), -v[1]*np.sin(theta)+v[2]*np.cos(theta))

def Rz(theta, v):
  '''Rotation of a vector around the z-axis'''
  return np.array(v[0]*np.cos(theta)+v[1]*np.sin(theta), -v[0]*np.sin(theta)+v[1]*np.cos(theta), v[2])

def R_L2I(v, pi, i, Omega):
  '''Rotation from the Lagrange to the Inertial RF'''
  return Rz( pi-Omega, Rx( i, Rz( Omega, v )))

def orbital_elements(planet_name, JD):
  '''Returns the Orbital elemnts at a given Julian date'''
  t = T(JD)
  T_pol = np.array([1,t,t**2,t**3]) #Polynomial in T (1, T, T^2, T^3)
  orbital_elements = {}
  for x in planet[planet_name].keys():
    orbital_elements[x] = np.dot(planet[planet_name][x], T_pol)

  return orbital_elements

def Lagrange_coord(planet_name, JD):
  '''Coordinates of the planet in the Lagrange RF'''
  OE = orbital_elements(planet_name, JD)
  a = OE['a']
  L = OE['L']
  e = OE['e']
